In [54]:
import pandas as pd
# column descriptions from https://www.retrosheet.org/gamelogs/glfields.txt
offensive_stats = ['AB', 'H', '2B', '3B', 'HR', 'RBI', 'SAC', 'SAC_FLY', 'HBP', 'BB', 'IBB', 'K', 'SB', 'CS', 'GDP', 'CI', 'LOB']
pitching_stats = ['pitchers_used', 'indv_earned_runs', 'team_earned_runs', 'wild_pitches', 'balks']
defensive_stats = ['putouts', 'assists', 'errors', 'passed_balls', 'double_plays', 'triple_plays']
roles = ['home_plate_ump', '1B_ump', '2B_ump', '3B_ump', 'LF_ump', 'RF_ump', 'away_manager', 'home_manager',
         'winning_pitcher', 'losing_pitcher', 'saving_pitcher', 'winning_rbi_batter', 'away_starting_pitcher', 'home_starting_pitcher']
columns = [
    'date',
    'series_game_num',
    'dow',
    'away_team',
    'away_team_league',
    'away_team_game_num',
    'home_team',
    'home_team_league',
    'home_team_game_num',
    'score_away',
    'score_home',
    'count_outs',
    'day_or_night',
    'makeup_info',
    'forfeit_info',
    'protest_info',
    'park_id',
    'attendance',
    'length_minutes',
    'away_line_score',
    'home_line_score',
    *['away_' + offensive_stat for offensive_stat in offensive_stats],
    *['away_' + pitching_stat for pitching_stat in pitching_stats],
    *['away_' + defensive_stat for defensive_stat in defensive_stats],
    *['home_' + offensive_stat for offensive_stat in offensive_stats],
    *['home_' + pitching_stat for pitching_stat in pitching_stats],
    *['home_' + defensive_stat for defensive_stat in defensive_stats],
    *list(itertools.chain.from_iterable([[role + '_id', role + '_name'] for role in roles])),
    *list(itertools.chain.from_iterable([[f'away_lineup{i}_id', f'away_lineup{i}_name', f'away_lineup{i}_pos'] for i in range(1, 10)])),
    *list(itertools.chain.from_iterable([[f'home_lineup{i}_id', f'home_lineup{i}_name', f'home_lineup{i}_pos'] for i in range(1, 10)])),
    'misc',
    'acquisition_info'
]

def file_to_df(fn):
    df = pd.read_csv(fn, header=None, names=columns)
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
    return df

In [71]:
import os
dfs = []
for fn in os.listdir('data'):
    year_df = file_to_df(f'data/{fn}')
    dfs.append(year_df)
    
df = pd.concat(dfs).sort_values('date')

In [86]:
# all 2019 yankees games
df[(df.date.dt.year == 2019) & ((df.home_team == 'NYA') | (df.away_team == 'NYA'))]

,date,series_game_num,dow,away_team,away_team_league,away_team_game_num,home_team,home_team_league,home_team_game_num,score_away,...,home_lineup7_name,home_lineup7_pos,home_lineup8_id,home_lineup8_name,home_lineup8_pos,home_lineup9_id,home_lineup9_name,home_lineup9_pos,misc,acquisition_info
11,2019-03-28,0,Thu,BAL,AL,1,NYA,AL,1,2,...,Greg Bird,3,torrg001,Gleyber Torres,4,tulot001,Troy Tulowitzki,6,NaN,Y
33,2019-03-30,0,Sat,BAL,AL,2,NYA,AL,2,5,...,Gleyber Torres,4,tulot001,Troy Tulowitzki,6,lemad001,DJ LeMahieu,5,NaN,Y
48,2019-03-31,0,Sun,BAL,AL,3,NYA,AL,3,7,...,Greg Bird,3,torrg001,Gleyber Torres,6,lemad001,DJ LeMahieu,4,NaN,Y
61,2019-04-01,0,Mon,DET,AL,5,NYA,AL,4,1,...,DJ LeMahieu,5,taucm001,Mike Tauchman,7,tulot001,Troy Tulowitzki,6,NaN,Y
73,2019-04-02,0,Tue,DET,AL,6,NYA,AL,5,3,...,Clint Frazier,10,wadet002,Tyler Wade,4,romia002,Austin Romine,2,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2355,2019-09-24,0,Tue,NYA,AL,158,TBA,AL,158,1,...,Joey Wendle,4,adamw002,Willy Adames,6,kierk001,Kevin Kiermaier,8,NaN,Y
2370,2019-09-25,0,Wed,NYA,AL,159,TBA,AL,159,0,...,Matt Duffy,5,kierk001,Kevin Kiermaier,8,adamw002,Willy Adames,6,NaN,Y
2396,2019-09-27,0,Fri,NYA,AL,160,TEX,AL,160,14,...,Scott Heineman,9,deshd002,Delino DeShields,8,trevj001,Jose Trevino,2,NaN,Y
2412,2019-09-28,0,Sat,NYA,AL,161,TEX,AL,161,4,...,Ronald Guzman,3,trevj001,Jose Trevino,2,kinei001,Isiah Kiner-Falefa,5,NaN,Y
